<a href="https://colab.research.google.com/github/kyou0612/Weeklyreport/blob/main/tfrecord.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.7.0


In [ ]:
import os
import numpy as np
import tensorflow as tf
import codecs
input_path = '/content/drive/MyDrive/experiment/learning_data/new/data_test_2014_MjT_tnsp_400000.txt'
# input_path = '/content/drive/MyDrive/experiment/learning_data/new/data_train_2013_MjT_tnsp_2000000.txt'
output_path = '/content/drive/MyDrive/experiment/learning_data/tfrecord/data_test_2014_MjT_p_24_36_1_'
# output_path = '/content/drive/MyDrive/experiment/learning_data/tfrecord/data_train_2013_MjT_n_24_36_1_'
x_feature_vector =[[[0] for j in range(36)] for i in range(96)]
# x_feature_vector =[[[0] for j in range(37)] for i in range(96)]
y = [[0] for i in range(4)]

# lichi = [0]*4
round = [0]*4 #東南西北
# t_round = 0 #東
# n_round = 0 #南
# s_round = 0 #西
# p_round = 0 #北
lastplayernum = 0

def make_feature_vector(line):
  # y = [0]
  line = line.split()
  
  xmk_round=[-1]*4
  global x_feature_vector
  global y 
  # global lichi
  global round 
  global lastplayernum
  jikaze = int(line[10].split(':')[1])
  playernum = jikaze - 27

  lichi = int(line[3].split(':')[1])
  # x_feature_vector[round[playernum]+playernum*24][36] = [lichi] 
  sute = line[6].split(':')[1].split(',')
  tsumogiri = line[7].split(':')[1].split(',')
  red  = line[8].split(':')[1].split(',')[-1]
  dora = [int(s.split(',')[0]) for s in line[0].split(':')[1:]]
  dora_round = [int(s.split(',')[1]) for s in line[0].split(':')[1:]]
  bakaze = int(line[9].split(':')[1])
  
  xmk_count = -1
  if len(sute) == 1 and playernum == 0:
    x_feature_vector =[[[0] for j in range(36)] for i in range(96)]
    # y = [0]
    # lichi = [0]*4
    round = [0]*4 #東南西北    
    lastplayernum = 0
    y = [[0] for i in range(4)]
    # for i in range(24):
    #   x_feature_vector[i][37] =[1]

  
  if sute != '':
    sute = [int(s) for s in sute]
    tsumogiri = [int(s) for s in tsumogiri]
    red = [int(s) for s in red]
    
    x_feature_vector[round[playernum]+playernum*24][(sute[-1] // 9) + 2] = [1]
    x_feature_vector[round[playernum]+playernum*24][(sute[-1] % 9) + 6] = [1]
    x_feature_vector[round[playernum]+playernum*24][tsumogiri[-1] + 19] = [1]
    x_feature_vector[round[playernum]+playernum*24][18] = [red[-1]]
    for j in range(len(dora)):
      if sute[-1] == dora[j] :
        x_feature_vector[round[playernum]+playernum*24][15] = [1]
      elif (sute[-1] == dora[j]+1 or sute[-1] == dora[j]-1) :
        x_feature_vector[round[playernum]+playernum*24][16] = [1]
      if sute[-1]//9 == dora[j]//9 :
        x_feature_vector[round[playernum]+playernum*24][17] = [1]
    if sute[-1] == bakaze:
      x_feature_vector[round[playernum]+playernum*24][1] = [1]
    if sute[-1] == jikaze:
      x_feature_vector[round[playernum]+playernum*24][0] = [1]

  
  


  naki_round = line[12].split(':')[1].split('],[')
  naki = [s.split(',') for s in  line[4].split(':')[1:]]
  yaochu = [0, 8, 9, 17, 18, 26, 27, 28, 29, 30, 31, 32, 33]
  # naki_xmk = []
  # for i in range(len(naki)-1,-1,-1):
  #   if naki[-1][0] == '5':
  #     naki_xmk.insert(0,naki[-1])
  #     naki.pop(i)
  while naki_round[-1] == '-1':
    naki_round.pop(-1)
    if naki_round == []:
      break
  
  
  # for i in range(len(naki_round)): 
  # print('new')
  # print(sute)
  # print(naki)
  # print(naki_round)
  # print('--------------')
  if naki_round != []: 
    for i in range(len(naki_round)):
      if naki_round[i].find(',') != -1:
        xmk_round[i]=int(naki_round[i].split(',')[1])
    max_xmk = max(xmk_round)#連カンは考えてない
    new_xmk = xmk_round.index(max_xmk)
    if  max_xmk != -1 and max_xmk==len(sute)-1:   
      xmk_count += 1
      naki_round[new_xmk] = [int(s) for s in naki_round[new_xmk].split(',')]
      if len(sute) == naki_round[new_xmk][1]+1:
        # print('xmk')
        
        if (playernum - lastplayernum)%4 == 2:
          for i in range(36):
            x_feature_vector[round[playernum-1]+(playernum-1)*24][i] = [-1]
          round[playernum-1]+=1
        elif (playernum - lastplayernum)%4 == 3:
          for i in range(36):
            x_feature_vector[round[playernum-1]+(playernum-1)*24][i] = [-1]
            x_feature_vector[round[playernum-2]+(playernum-2)*24][i] = [-1]
          round[playernum-1]+=1
          round[playernum-2]+=1



        x_feature_vector[round[playernum]+playernum*24][26] = [1]
        x_feature_vector[round[playernum]+playernum*24][21] = [1]
        x_feature_vector[round[playernum]+playernum*24][20] = [0]
        x_feature_vector[round[playernum]+playernum*24][19] = [0]

        x_feature_vector[round[playernum]+playernum*24][(int(naki[new_xmk][1])//9)+27] = [1]
        # x_feature_vector[naki_round[-1][0]][23] = [1]
        # x_feature_vector[naki_round[-1][0]][21] = [1]
        # x_feature_vector[naki_round[-1][0]][20] = [0]
        # x_feature_vector[naki_round[-1][0]][19] = [0]
        # x_feature_vector[naki_round[-1][0]][(int(naki[-1][1])//9)+27] = [1]
        if int(naki[new_xmk][1]) in dora:
          x_feature_vector[round[playernum]+playernum*24][31] = [1]
          
        if int(naki[new_xmk][1]) in yaochu:
          x_feature_vector[round[playernum]+playernum*24][32] = [1]
          
        if int(naki[new_xmk][1]) == jikaze:
          x_feature_vector[round[playernum]+playernum*24][33] = [1]
          
        if int(naki[new_xmk][1]) == bakaze:
          x_feature_vector[round[playernum]+playernum*24][34] = [1]
          
        if int(naki[new_xmk][1]) >= 31:
          x_feature_vector[round[playernum]+playernum*24][35] = [1]
      
    else:
      if naki_round[-1].find(',') == -1:
        naki_round[-1] = int(naki_round[-1])
        if len(sute) == naki_round[-1]+1: 
          if (playernum - lastplayernum)%4 == 2:
            for i in range(36):
              x_feature_vector[round[playernum-1]+(playernum-1)*24][i] = [-1]
            round[playernum-1]+=1
          if (playernum - lastplayernum)%4 == 3:
            for i in range(36):
              x_feature_vector[round[playernum-1]+(playernum-1)*24][i] = [-1]
              x_feature_vector[round[playernum-2]+(playernum-2)*24][i] = [-1]
            round[playernum-1]+=1
            round[playernum-2]+=1                                       
          x_feature_vector[round[playernum]+playernum*24][21] = [1]
          x_feature_vector[round[playernum]+playernum*24][20] = [0]
          x_feature_vector[round[playernum]+playernum*24][19] = [0]
          x_feature_vector[round[playernum]+playernum*24][(int(naki[-1][0]))+21] = [1]
          x_feature_vector[round[playernum]+playernum*24][(int(naki[-1][1])//9)+27] = [1]
          if int(naki[-1][0]) == 1:
            if int(naki[-1][1]) in dora or (int(naki[-1][2])) in dora or (int(naki[-1][3])) in dora:
              x_feature_vector[round[playernum]+playernum*24][31] = [1]
            if int(naki[-1][1]) in yaochu or (int(naki[-1][2])) in yaochu or (int(naki[-1][3])) in yaochu:
              x_feature_vector[round[playernum]+playernum*24][32] = [1]
          else:
            if int(naki[-1][1]) in dora:
              x_feature_vector[round[playernum]+playernum*24][31] = [1]
            if int(naki[-1][1]) in yaochu:
              x_feature_vector[round[playernum]+playernum*24][32] = [1]
            if int(naki[-1][1]) == jikaze:
              x_feature_vector[round[playernum]+playernum*24][33] = [1]
            if int(naki[-1][1]) == bakaze:
              x_feature_vector[round[playernum]+playernum*24][34] = [1]
            if int(naki[-1][1]) >= 31:
              x_feature_vector[round[playernum]+playernum*24][35] = [1]



  xiangting = int(line[2].split(':')[1])
  # print('%d,%d'%(xiangting,playernum))
  
  # if playernum == 0:
    
    # print(x_feature_vector[round[playernum]+playernum*24][37])

  # if xiangting == 0 :
  #   y = [1]
  #   # print('yes')
  if xiangting == 0:
    y[playernum] = [1]
  else:
    y[playernum] = [0]

  round[playernum]+=1
  # print(round)
  # print(lastplayernum)
  # print(playernum)
  
  lastplayernum = playernum 
  # if playernum == 0:
  #   x_feature_vector1 =x_feature_vector[0:24]
  #   x_feature_vector2 =x_feature_vector[24:48]
  #   x_feature_vector3 =x_feature_vector[48:72]
  #   x_feature_vector4 =x_feature_vector[72:96]
  #   y1 = y[0].copy()
  #   y2 = y[1].copy()
  #   y3 = y[2].copy()
  #   y4 = y[3].copy()
  # elif playernum == 1:
  #   x_feature_vector1 =x_feature_vector[24:48]
  #   x_feature_vector2 =x_feature_vector[48:72]
  #   x_feature_vector3 =x_feature_vector[72:96]  
  #   x_feature_vector4 =x_feature_vector[0:24]
  #   y1 = y[1].copy()
  #   y2 = y[2].copy()
  #   y3 = y[3].copy()
  #   y4 = y[0].copy()
  # elif playernum == 2:
  #   x_feature_vector1 =x_feature_vector[48:72]
  #   x_feature_vector2 =x_feature_vector[72:96] 
  #   x_feature_vector3 =x_feature_vector[0:24]
  #   x_feature_vector4 =x_feature_vector[24:48]
  #   y1 = y[2].copy()
  #   y2 = y[3].copy()
  #   y3 = y[0].copy()
  #   y4 = y[1].copy()
  # elif playernum == 3:
  #   x_feature_vector1 =x_feature_vector[72:96]
  #   x_feature_vector2 =x_feature_vector[0:24]
  #   x_feature_vector3 =x_feature_vector[24:48]
  #   x_feature_vector4 =x_feature_vector[48:72]
  #   y1 = y[3].copy()
  #   y2 = y[0].copy()
  #   y3 = y[1].copy()
  #   y4 = y[2].copy()

  # x_feature_vector_all= x_feature_vector1 + x_feature_vector2 + x_feature_vector3 + x_feature_vector4
  
  xfv1=[[[0] for j in range(36)] for i in range(96)]
  for j in range(96):
    for k in range(36):
      xfv1[j][k]=x_feature_vector[j][k].copy()
  y1 = y[0].copy()
  y2 = y[1].copy()
  y3 = y[2].copy()
  y4 = y[3].copy()

  sutelen = len(sute)
  return xfv1 ,y1,y2,y3,y4 ,lichi ,playernum ,sutelen  

def make_feature_vector_forone(line): 
  # y = [0]
  line = line.split()
  
  xmk_round=[-1]*4
  global x_feature_vector
  global y 
  # global lichi
  global round 
  global lastplayernum
  jikaze = int(line[10].split(':')[1])
  playernum = jikaze - 27

  lichi = int(line[3].split(':')[1])
  # x_feature_vector[round[playernum]+playernum*24][36] = [lichi] 
  sute = line[6].split(':')[1].split(',')
  tsumogiri = line[7].split(':')[1].split(',')
  red  = line[8].split(':')[1].split(',')[-1]
  dora = [int(s.split(',')[0]) for s in line[0].split(':')[1:]]
  dora_round = [int(s.split(',')[1]) for s in line[0].split(':')[1:]]
  bakaze = int(line[9].split(':')[1])
  
  xmk_count = -1
  if len(sute) == 1 and playernum == 0:
    x_feature_vector =[[[0] for j in range(36)] for i in range(96)]
    # y = [0]
    # lichi = [0]*4
    round = [0]*4 #東南西北    
    lastplayernum = 0
    y = [[0] for i in range(4)]
    # for i in range(24):
    #   x_feature_vector[i][37] =[1]

  
  if sute != '':
    sute = [int(s) for s in sute]
    tsumogiri = [int(s) for s in tsumogiri]
    red = [int(s) for s in red]
    
    x_feature_vector[round[playernum]+playernum*24][(sute[-1] // 9) + 2] = [1]
    x_feature_vector[round[playernum]+playernum*24][(sute[-1] % 9) + 6] = [1]
    x_feature_vector[round[playernum]+playernum*24][tsumogiri[-1] + 19] = [1]
    x_feature_vector[round[playernum]+playernum*24][18] = [red[-1]]
    for j in range(len(dora)):
      if sute[-1] == dora[j] :
        x_feature_vector[round[playernum]+playernum*24][15] = [1]
      elif (sute[-1] == dora[j]+1 or sute[-1] == dora[j]-1) :
        x_feature_vector[round[playernum]+playernum*24][16] = [1]
      if sute[-1]//9 == dora[j]//9 :
        x_feature_vector[round[playernum]+playernum*24][17] = [1]
    if sute[-1] == bakaze:
      x_feature_vector[round[playernum]+playernum*24][1] = [1]
    if sute[-1] == jikaze:
      x_feature_vector[round[playernum]+playernum*24][0] = [1]

  
  


  naki_round = line[12].split(':')[1].split('],[')
  naki = [s.split(',') for s in  line[4].split(':')[1:]]
  yaochu = [0, 8, 9, 17, 18, 26, 27, 28, 29, 30, 31, 32, 33]
  # naki_xmk = []
  # for i in range(len(naki)-1,-1,-1):
  #   if naki[-1][0] == '5':
  #     naki_xmk.insert(0,naki[-1])
  #     naki.pop(i)
  while naki_round[-1] == '-1':
    naki_round.pop(-1)
    if naki_round == []:
      break
  
  
  # for i in range(len(naki_round)): 
  # print('new')
  # print(sute)
  # print(naki)
  # print(naki_round)
  # print('--------------')
  if naki_round != []: 
    for i in range(len(naki_round)):
      if naki_round[i].find(',') != -1:
        xmk_round[i]=int(naki_round[i].split(',')[1])
    max_xmk = max(xmk_round)#連カンは考えてない
    new_xmk = xmk_round.index(max_xmk)
    if  max_xmk != -1 and max_xmk==len(sute)-1:   
      xmk_count += 1
      naki_round[new_xmk] = [int(s) for s in naki_round[new_xmk].split(',')]
      if len(sute) == naki_round[new_xmk][1]+1:
        # print('xmk')
        
        # if (playernum - lastplayernum)%4 == 2:
        #   for i in range(36):
        #     x_feature_vector[round[playernum-1]+(playernum-1)*24][i] = [-1]
        #   round[playernum-1]+=1
        # elif (playernum - lastplayernum)%4 == 3:
        #   for i in range(36):
        #     x_feature_vector[round[playernum-1]+(playernum-1)*24][i] = [-1]
        #     x_feature_vector[round[playernum-2]+(playernum-2)*24][i] = [-1]
        #   round[playernum-1]+=1
        #   round[playernum-2]+=1



        x_feature_vector[round[playernum]+playernum*24][26] = [1]
        x_feature_vector[round[playernum]+playernum*24][21] = [1]
        x_feature_vector[round[playernum]+playernum*24][20] = [0]
        x_feature_vector[round[playernum]+playernum*24][19] = [0]

        x_feature_vector[round[playernum]+playernum*24][(int(naki[new_xmk][1])//9)+27] = [1]
        # x_feature_vector[naki_round[-1][0]][23] = [1]
        # x_feature_vector[naki_round[-1][0]][21] = [1]
        # x_feature_vector[naki_round[-1][0]][20] = [0]
        # x_feature_vector[naki_round[-1][0]][19] = [0]
        # x_feature_vector[naki_round[-1][0]][(int(naki[-1][1])//9)+27] = [1]
        if int(naki[new_xmk][1]) in dora:
          x_feature_vector[round[playernum]+playernum*24][31] = [1]
          
        if int(naki[new_xmk][1]) in yaochu:
          x_feature_vector[round[playernum]+playernum*24][32] = [1]
          
        if int(naki[new_xmk][1]) == jikaze:
          x_feature_vector[round[playernum]+playernum*24][33] = [1]
          
        if int(naki[new_xmk][1]) == bakaze:
          x_feature_vector[round[playernum]+playernum*24][34] = [1]
          
        if int(naki[new_xmk][1]) >= 31:
          x_feature_vector[round[playernum]+playernum*24][35] = [1]
      
    else:
      if naki_round[-1].find(',') == -1:
        naki_round[-1] = int(naki_round[-1])
        if len(sute) == naki_round[-1]+1: 
          # if (playernum - lastplayernum)%4 == 2:
          #   for i in range(36):
          #     x_feature_vector[round[playernum-1]+(playernum-1)*24][i] = [-1]
          #   round[playernum-1]+=1
          # if (playernum - lastplayernum)%4 == 3:
          #   for i in range(36):
          #     x_feature_vector[round[playernum-1]+(playernum-1)*24][i] = [-1]
          #     x_feature_vector[round[playernum-2]+(playernum-2)*24][i] = [-1]
          #   round[playernum-1]+=1
          #   round[playernum-2]+=1                                       
          x_feature_vector[round[playernum]+playernum*24][21] = [1]
          x_feature_vector[round[playernum]+playernum*24][20] = [0]
          x_feature_vector[round[playernum]+playernum*24][19] = [0]
          x_feature_vector[round[playernum]+playernum*24][(int(naki[-1][0]))+21] = [1]
          x_feature_vector[round[playernum]+playernum*24][(int(naki[-1][1])//9)+27] = [1]
          if int(naki[-1][0]) == 1:
            if int(naki[-1][1]) in dora or (int(naki[-1][2])) in dora or (int(naki[-1][3])) in dora:
              x_feature_vector[round[playernum]+playernum*24][31] = [1]
            if int(naki[-1][1]) in yaochu or (int(naki[-1][2])) in yaochu or (int(naki[-1][3])) in yaochu:
              x_feature_vector[round[playernum]+playernum*24][32] = [1]
          else:
            if int(naki[-1][1]) in dora:
              x_feature_vector[round[playernum]+playernum*24][31] = [1]
            if int(naki[-1][1]) in yaochu:
              x_feature_vector[round[playernum]+playernum*24][32] = [1]
            if int(naki[-1][1]) == jikaze:
              x_feature_vector[round[playernum]+playernum*24][33] = [1]
            if int(naki[-1][1]) == bakaze:
              x_feature_vector[round[playernum]+playernum*24][34] = [1]
            if int(naki[-1][1]) >= 31:
              x_feature_vector[round[playernum]+playernum*24][35] = [1]



  xiangting = int(line[2].split(':')[1])
  # print('%d,%d'%(xiangting,playernum))
  
  # if playernum == 0:
    
    # print(x_feature_vector[round[playernum]+playernum*24][37])

  # if xiangting == 0 :
  #   y = [1]
  #   # print('yes')
  if xiangting == 0:
    y[playernum] = [1]
  else:
    y[playernum] = [0]

  round[playernum]+=1
  # print(round)
  # print(lastplayernum)
  # print(playernum)
  
  lastplayernum = playernum 
  if playernum == 0:
    x_feature_vector1 =x_feature_vector[0:24]
  #   x_feature_vector2 =x_feature_vector[24:48]
  #   x_feature_vector3 =x_feature_vector[48:72]
  #   x_feature_vector4 =x_feature_vector[72:96]
    y1 = y[0].copy()
  #   y2 = y[1].copy()
  #   y3 = y[2].copy()
  #   y4 = y[3].copy()
  elif playernum == 1:
    x_feature_vector1 =x_feature_vector[24:48]
  #   x_feature_vector2 =x_feature_vector[48:72]
  #   x_feature_vector3 =x_feature_vector[72:96]  
  #   x_feature_vector4 =x_feature_vector[0:24]
    y1 = y[1].copy()
  #   y2 = y[2].copy()
  #   y3 = y[3].copy()
  #   y4 = y[0].copy()
  elif playernum == 2:
    x_feature_vector1 =x_feature_vector[48:72]
  #   x_feature_vector2 =x_feature_vector[72:96] 
  #   x_feature_vector3 =x_feature_vector[0:24]
  #   x_feature_vector4 =x_feature_vector[24:48]
    y1 = y[2].copy()
  #   y2 = y[3].copy()
  #   y3 = y[0].copy()
  #   y4 = y[1].copy()
  elif playernum == 3:
    x_feature_vector1 =x_feature_vector[72:96]
  #   x_feature_vector2 =x_feature_vector[0:24]
  #   x_feature_vector3 =x_feature_vector[24:48]
  #   x_feature_vector4 =x_feature_vector[48:72]
    y1 = y[3].copy()
  #   y2 = y[0].copy()
  #   y3 = y[1].copy()
  #   y4 = y[2].copy()

  # x_feature_vector_all= x_feature_vector1 + x_feature_vector2 + x_feature_vector3 + x_feature_vector4
  
  xfv1=[[[0] for j in range(36)] for i in range(24)]
  for j in range(24):
    for k in range(36):
      xfv1[j][k]=x_feature_vector1[j][k].copy()
  

  sutelen = len(sute)
  return xfv1 ,y1 ,lichi ,playernum ,sutelen                                 
                                          


  
                                                                                    
def read_data(fin,batch_size):
  x1_list, y1_list,y2_list,y3_list,y4_list,sutelen_list,lichi_list,pn_list = [],[],[],[],[],[],[],[]
  eofeof = False
  count = 0
  y_count = 0
  
  
  for i in range(batch_size):
    line = fin.readline().replace('\n', '')
    # count+=1
    if line:
      pass
    else:
      eofeof = True
      break
    # x1,y1,y2,y3,y4,lichi,pn,sutelen = make_feature_vector(line)
    x1,y1,lichi,pn,sutelen = make_feature_vector_forone(line)
    if lichi == 1 :              
      continue

    if pn != 3:
      continue

    if y1 == [1]:
      y_count +=1

    
  
    
    count +=1
    x1_list.append(x1)
    # x2_list.append(x2)
    # x3_list.append(x3)
    # x4_list.append(x4)
    y1_list.append(y1) 
    # y2_list.append(y2)
    # y3_list.append(y3)
    # y4_list.append(y4) 
    lichi_list.append(lichi)
    pn_list.append(pn)

    sutelen_list.append(sutelen)
    if count %10000==0:
      print(count)
      

    
  print('yes')
  x1_list = np.array(x1_list)
  # x2_list = np.array(x2_list)
  # x3_list = np.array(x3_list)
  # x4_list = np.array(x4_list)
  y1_list = np.array(y1_list)
  # y2_list = np.array(y2_list)
  # y3_list = np.array(y3_list)
  # y4_list = np.array(y4_list)
  lichi_list = np.array(lichi_list)
  pn_list = np.array(pn_list)  
  sutelen_list = np.array(sutelen_list)
  # pre_1 = y_count/count
  # print(pre_1)
  print(y_count)
  print(count)



  # return x1_list, y1_list,y2_list,y3_list,y4_list, eofeof,sutelen_list,lichi_list,pn_list
  return x1_list, y1_list, eofeof,sutelen_list,lichi_list,pn_list


 
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# def serialize_sample(x, y1,y2,y3,y4,sutelen,lichi,pn):
def serialize_sample(x, y1,sutelen,lichi,pn):
    x_binary = (x.astype(np.int32)).tobytes()
    y1_binary = (y1.astype(np.int32)).tobytes()
    # y2_binary = (y2.astype(np.int32)).tobytes()
    # y3_binary = (y3.astype(np.int32)).tobytes()
    # y4_binary = (y4.astype(np.int32)).tobytes()
    sutelen_binary = (sutelen.astype(np.int32)).tobytes()
    lichi_binary = (lichi.astype(np.int32)).tobytes()
    pn_binary = (pn.astype(np.int32)).tobytes()
    x_list = _bytes_feature(x_binary)
    y1_list = _bytes_feature(y1_binary)
    # y2_list = _bytes_feature(y2_binary)
    # y3_list = _bytes_feature(y3_binary)
    # y4_list = _bytes_feature(y4_binary)
    sutelen_list = _bytes_feature(sutelen_binary)
    lichi_list = _bytes_feature(lichi_binary)
    pn_list = _bytes_feature(pn_binary)

    proto = tf.train.Example(features=tf.train.Features(feature={
        "x": x_list, # int32, (96, 37, 1)
        "y1": y1_list, # int32, (1, )
        # "y2": y2_list, # int32, (1, )
        # "y3": y3_list, # int32, (1, )
        # "y4": y4_list, # int32, (1, )
        "sutelen": sutelen_list,
        "lichi": lichi_list,
        "playernumber": pn_list
    }))
    return proto.SerializeToString()

def write_record():
    fcount=1
    with codecs.open(input_path, 'r', 'utf-8', 'ignore') as fin:
          eofeof = False
          while not eofeof:
            x = []
            y1 = []
            # y2 = []
            # y3 = []
            # y4 = []
            sutelen = []
            lichi = []
            pn = []
            # x , y1 ,y2 ,y3 ,y4 , eofeof,sutelen,lichi,pn = read_data(fin,50000)   
            x , y1  , eofeof,sutelen,lichi,pn = read_data(fin,200000)   
            print(len(sutelen)) 
            print(len(x))    
            print(pn[90:100])     
            with tf.io.TFRecordWriter(output_path + str(fcount)+'.tfrecords') as writer:
                for i in range(x.shape[0]):
                    # example = serialize_sample(x[i], y1[i], y2[i], y3[i], y4[i],sutelen[i],lichi[i],pn[i])
                    example = serialize_sample(x[i], y1[i], sutelen[i], lichi[i], pn[i])
                    writer.write(example)
            fcount+=1


# 書き込み

write_record()


In [ ]:
# データ解析
def deserialize_example(serialized_string):
    image_feature_description = {
        'x': tf.io.FixedLenFeature([], tf.string),
        'y': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(serialized_string, image_feature_description)
    image = tf.reshape(tf.io.decode_raw(example["x"], tf.int32), (24, 36, 1))
    label = tf.io.decode_raw(example["y"], tf.int32)
    return image, label

def read_record():
    fcount = 1
    dataset = tf.data.TFRecordDataset(output_path+ str(fcount)+'.tfrecords').map(deserialize_example).batch(100000)

    for x in dataset:
        print(x[1])
        break



read_record()